In [86]:
#!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cpu.html
#!pip install torch_geometric
#!pip install pyvista
#%pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.0.1+cpu.html

In [87]:
import torch
import matplotlib.pyplot as plt
from utils import DelaunayTransform
from torch_geometric.data import Data
import networkx as nx
import pyvista as pv
import numpy as np
torch.__version__

'2.0.1'

In [88]:
import random
N = 150_000
pos = torch.rand((N, 2))
data = Data(pos=pos, surf=torch.full((N, 1), False))
transform = DelaunayTransform()
data = transform(data)
data.pos = pos

#data = torch.load('./sampleData.pth')

def plot_graph(data, l=1, plotter=None):
    c = ['r', 'g', 'b', 'm']
    
    p = pv.Plotter() if plotter is None else plotter
    
    mesh = pv.PolyData()
    if data.pos.shape[1] != 3:
        mesh.points = np.concatenate([data.pos.numpy(), np.full((data.pos.shape[0], 1), l)], axis=1) 
    else:
        mesh.points = data.pos.numpy()
    edges = data.edge_index.t().numpy()
    lines = np.hstack([np.full((edges.shape[0], 1), 2), edges]).ravel()
    mesh.lines = lines
    p.add_mesh(mesh, line_width=1, color=random.choice(c))
    
    if plotter is None:
        p.show()

In [89]:
#plot_graph(data)

In [90]:
import time

def divide_mesh(v, e, k):
    clusters = [Data(edge_ids=set()) for _ in range(k)]
    
    # Randomly initialize centroids (2D points)
    centroids = torch.rand((k, 2), device=v.device)

    # Precompute edge directions and norms
    edges_directions = v[e[:, 1]] - v[e[:, 0]]
    edges_norms = torch.norm(edges_directions, dim=1, keepdim=True)  # Shape: [num_edges, 1]
    edges_directions /= edges_norms  # Normalize edge directions
    start_all = time.time()
    norm_changes = float('inf')
    while norm_changes > 1e-3:
        # Vectorized clustering step
        centroids_norms = torch.norm(centroids, dim=1, keepdim=True)  # Shape: [num_centroids, 1]
        cosine_angles = torch.matmul(edges_directions, centroids.T) / (centroids_norms.T)  # Shape: [num_edges, num_centroids]
        angles = torch.acos(cosine_angles)  # Ensure values are in valid range for acos
        min_edge_idxs = torch.argmin(angles, dim=1)  # Shape: [num_edges]
        # Efficient assignment to clusters using torch
        cluster_masks = [(min_edge_idxs == i) for i in range(k)]
        for i in range(k):
            clusters[i].edge_ids.update(torch.nonzero(cluster_masks[i]).squeeze(1).tolist())

        # Efficient centroid update
        n_m = 0.0
        for i in range(k):
            if clusters[i].edge_ids:  # Check if the cluster has assigned edges
                cluster_edges = edges_directions[torch.tensor(list(clusters[i].edge_ids), device=v.device)]
                last_centroid = centroids[i].clone()
                centroids[i] = torch.mean(cluster_edges, dim=0)
                n_m = max(torch.norm(centroids[i] - last_centroid), n_m)
        norm_changes = n_m

    print("All:", time.time() - start_all)

    # Post-process clusters to finalize edge indices
    for cluster in clusters:
        cluster.edge_index = e[list(cluster.edge_ids)]
        del cluster.edge_ids

    return clusters


device = torch.device('cpu')
data.pos = data.pos[:, :2].to(device) 
clusters = divide_mesh(data.pos, data.edge_index.T, 8)
clusters

All: 1.8245131969451904


[Data(edge_index=[80305, 2]),
 Data(edge_index=[202314, 2]),
 Data(edge_index=[70922, 2]),
 Data(edge_index=[32902, 2]),
 Data(edge_index=[29803, 2]),
 Data(edge_index=[14078, 2]),
 Data(edge_index=[195270, 2]),
 Data(edge_index=[13520, 2])]

In [92]:
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.cluster import MiniBatchKMeans
# If you have a GPU, consider using cuML's KMeans
# from cuml.cluster import KMeans

def generate_coarse_graph_with_clustering(g, num_clusters):
    """
    Generates a coarser graph using vectorized operations and optimized clustering.
    """
    pos = g.pos.cpu().numpy()
    edges = g.edge_index.cpu().numpy()
    
    # Use MiniBatchKMeans with an optimized batch size
    kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=100000, random_state=0)
    kmeans.fit(pos)
    labels = kmeans.labels_
    cluster_centers = kmeans.cluster_centers_
    
    # Vectorized mapping of edges to cluster labels
    cluster_labels = labels.astype(np.int64)
    cluster_edges = np.vstack((cluster_labels[edges[0]], cluster_labels[edges[1]]))
    
    # Remove self-loops and duplicate edges
    mask = cluster_edges[0] != cluster_edges[1]
    cluster_edges = cluster_edges[:, mask]
    cluster_edges = np.unique(np.sort(cluster_edges, axis=0), axis=1)
    
    new_edges = torch.tensor(cluster_edges, dtype=torch.long)
    new_pos = torch.tensor(cluster_centers, dtype=torch.float)
    
    coarse_graph = Data(pos=new_pos, edge_index=new_edges)
    return coarse_graph, labels

# Generate the hierarchical mesh
graphs = [data]
mappings = []
current_graph = data
num_levels = 3

for level in range(num_levels):
    num_clusters = max(2, int(len(current_graph.pos) // 10))
    print(f"Generating coarse graph at level {level+1} with {num_clusters} clusters...")
    coarse_graph, mapping = generate_coarse_graph_with_clustering(current_graph, num_clusters)
    graphs.append(coarse_graph)
    mappings.append(mapping)
    current_graph = coarse_graph

# Construct the final multi-layer graph
all_positions = []
all_edges = []
node_offsets = []

offset = 0
z_offset = 1.0

for layer_index, graph in enumerate(graphs):
    num_nodes = graph.pos.shape[0]
    pos = graph.pos

    # Add vertical dimension for visualization
    if pos.shape[1] == 2:
        z = torch.full((num_nodes, 1), layer_index * z_offset)
        pos = torch.cat((pos, z), dim=1)
    else:
        pos[:, -1] += layer_index * z_offset

    all_positions.append(pos)
    node_offsets.append(offset)

    if graph.edge_index.numel() > 0:
        adjusted_edges = graph.edge_index + offset
        all_edges.append(adjusted_edges)

    offset += num_nodes

# Create inter-layer edges based on the mappings
for level, mapping in enumerate(mappings):
    start_idx = node_offsets[level]
    next_layer_start_idx = node_offsets[level+1]
    num_nodes = len(mapping)
    
    node_indices = np.arange(num_nodes) + start_idx
    cluster_indices = mapping + next_layer_start_idx
    inter_layer_edges = np.vstack((node_indices, cluster_indices))
    all_edges.append(torch.tensor(inter_layer_edges, dtype=torch.long))

# Combine all positions and edges
all_positions = torch.cat(all_positions, dim=0)
all_edges = torch.cat(all_edges, dim=1)

final_graph = Data(pos=all_positions, edge_index=all_edges)

# Plot the final graph
p = pv.Plotter()
plot_graph(final_graph, 1, p)
p.show()


Generating coarse graph at level 1 with 15000 clusters...
